# Setup

 Import standard modules

In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
import numpy as np
import pandas as pd
import time
os.chdir('/home/chris/projects/tidal_flat_0d')

In [2]:
from src import features, models
from pyprojroot import here
root = here()
sys.path.append(str(root))
wdir = root / "data" / "interim"

Import project modules from src folder

In [3]:
tides = features.load_tide(wdir, "tides.feather")
tides = tides - (np.mean(tides) + 0.6)
# tides = tides.iloc[:int(len(tides) * 0.35)]

In [4]:
from src import models
import cProfile

flat = models.TidalFlat(
    tide_ts=tides,
    land_elev_init=1.15,
    conc_init=0.0,
    conc_bound=1.0,
    grain_diam=3.5e-5,
    grain_dens=2.65e3,
    bulk_dens=900,
    org_rate=2e-4,
    comp_rate=-4e-3,
    sub_rate=-3e-3,
    slr=0.005,
    solver_method="DOP853",
    years=1,
    dense_output=True,
)
cProfile.run("flat.simulate()", "flat_stats")

Progress: 185.0inundation [02:52,  1.07inundation/s]                   

----------------------------------------
Starting elevation:        1.150      m
Final elevation:           1.179      m
Elevation change:          2.852     cm
----------------------------------------
Aggradation:               3.502     cm
Degradation:              0.620      cm
----------------------------------------
Runtime:                      00:02:52


In [17]:
import pstats
from pstats import SortKey
p = pstats.Stats('flat_stats')
p.sort_stats(SortKey.CUMULATIVE).print_stats(20)

Mon May 17 17:43:18 2021    flat_stats

         16745060 function calls (16702327 primitive calls) in 172.773 seconds

   Ordered by: cumulative time
   List reduced from 1025 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    377/1    0.031    0.000  172.774  172.774 {built-in method builtins.exec}
        1    0.000    0.000  172.774  172.774 <string>:1(<module>)
        1    0.000    0.000  172.774  172.774 /home/chris/projects/tidal_flat_0d/src/models.py:656(simulate)
        1    0.010    0.010  172.766  172.766 /home/chris/projects/tidal_flat_0d/src/models.py:615(run)
      186    0.192    0.001  172.756    0.929 /home/chris/projects/tidal_flat_0d/src/models.py:601(step)
740081/712261   93.375    0.000   96.928    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
   134302    0.172    0.000   92.374    0.001 <__array_function__ internals>:2(concatenate)
      186    0.522    0.003   76.242    0.4